In [1]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
import random
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objs as go

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler, RobustScaler

# 난수 생성 고정
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# 데이터 경로 설정
DATA_PATH = Path("./data")
train_data = pd.read_csv(DATA_PATH / "train.csv")
test_data = pd.read_csv(DATA_PATH / "test.csv")

# 필요 피처 선택
non_numeric_cols = ["faultNumber", "simulationRun", "sample"]
use_cols = train_data.columns.difference(non_numeric_cols)

X_train = train_data[use_cols]
X_test = test_data[use_cols]

# 데이터 스케일러 인스턴스 생성(데이터 표준화)
# scaler = MinMaxScaler()
scaler = RobustScaler()
# 학습 데이터셋에 대해 fit과 transform 수행: train 기준 정보 계산 및 데이터 변환
X_train_scaled = scaler.fit_transform(X_train) 
# 테스트 데이터셋에 대해서는 transform만 수행: 학습 데이터셋의 기준 정보를 사용하여 데이터 변환
X_test_scaled = scaler.transform(X_test)

# PyTorch Tensor로 변환 
X_train_tensor = torch.FloatTensor(X_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)

In [2]:
print(type(X_train))
print(X_train.shape) 

print(type(X_train_scaled))
print(X_train_scaled.shape)

X_train_tensor = torch.FloatTensor(X_train_scaled)
print(type(X_train_tensor))
print(X_train_tensor.shape) 


<class 'pandas.core.frame.DataFrame'>
(250000, 52)
<class 'numpy.ndarray'>
(250000, 52)
<class 'torch.Tensor'>
torch.Size([250000, 52])


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class VAE(nn.Module):
    def __init__(self, input_dim=52, hidden_dim=128, latent_dim=20):
        super(VAE, self).__init__()
        
        # 인코더
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)  # 평균
        self.fc22 = nn.Linear(hidden_dim, latent_dim)  # 로그 분산
        
        # 디코더
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)
    
    def encode(self, x):
        h1 = torch.relu(self.fc1(x))
        mu = self.fc21(h1)
        logvar = self.fc22(h1)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        h3 = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))  # Sigmoid 활성화 함수 사용
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# 손실 함수 정의
def loss_function(recon_x, x, mu, logvar):
    recon_loss = nn.BCELoss(reduction='sum')(recon_x, x)  # 재구성 손실
    kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  # KL 발산
    return recon_loss + kld_loss

# 데이터 준비
# TensorDataset 및 DataLoader로 데이터 배치 처리
dataset = TensorDataset(X_train_tensor)  # 텐서 데이터셋 생성
batch_size = 64  # 원하는 배치 크기로 설정
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 모델, 손실 함수 및 옵티마이저 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE(input_dim=52).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 학습 과정
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    train_loss = 0
    for batch_idx, batch_data in enumerate(train_loader):
        # 만약 batch_data가 리스트라면 첫 번째 항목이 실제 텐서 데이터임
        if isinstance(batch_data, list):
            batch_data = batch_data[0]
        
        data = batch_data.to(device)
        optimizer.zero_grad()
        
        # VAE를 통한 이상 탐지 모델 학습
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    print(f'Epoch {epoch + 1}, Loss: {train_loss / len(train_loader.dataset):.4f}')

print("Training finished.")

/opt/conda/conda-bld/pytorch_1728929546833/work/aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [4,0,0], thread: [66,0,0] Assertion `target_val >= zero && target_val <= one` failed.
/opt/conda/conda-bld/pytorch_1728929546833/work/aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [4,0,0], thread: [68,0,0] Assertion `target_val >= zero && target_val <= one` failed.
/opt/conda/conda-bld/pytorch_1728929546833/work/aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [4,0,0], thread: [69,0,0] Assertion `target_val >= zero && target_val <= one` failed.
/opt/conda/conda-bld/pytorch_1728929546833/work/aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [4,0,0], thread: [71,0,0] Assertion `target_val >= zero && target_val <= one` failed.
/opt/conda/conda-bld/pytorch_1728929546833/work/aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [4,0,0], thread: [73,0,0] Assertion `target_val >= zero && target_val <= one` failed.
/opt/conda/conda-bld/pytorch_1728929546833/wo

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [4]:
# 이상 탐지를 위한 임계값 설정 및 이상 여부 판단
def detect_anomaly(data, model, threshold):
    model.eval()
    with torch.no_grad():
        recon, _, _ = model(data.to(device))
        recon_error = torch.mean((recon - data.to(device)) ** 2, dim=1)  # MSE 계산
        return recon_error > threshold  # 재구성 오차가 임계값을 넘는 경우 이상으로 판단

# 정상 데이터에서 추정한 임계값을 설정
normal_data = X_train_tensor
recon_errors = []
for data in normal_data:
    recon, _, _ = model(data.unsqueeze(0).to(device))
    error = torch.mean((recon - data.unsqueeze(0).to(device)) ** 2).item()
    recon_errors.append(error)

# 임계값을 정상 데이터의 평균 오차 + 3*표준편차로 설정
threshold = torch.tensor(recon_errors).mean() + 3 * torch.tensor(recon_errors).std()

# 새로운 데이터로 이상 탐지 수행
test_data = X_test_tensor.to(device) # 테스트 데이터 (정상 또는 이상)
is_anomaly = detect_anomaly(test_data, model, threshold)

In [5]:
# is_anomaly를 0과 1로 변환
is_anomaly_numeric = is_anomaly.int()  # True는 1, False는 0으로 변환

# NumPy 배열로 변환
is_anomaly_array = is_anomaly_numeric.cpu().numpy()  # CPU로 이동 후 NumPy로 변환

# DataFrame 생성
df_output = pd.DataFrame(is_anomaly_array, columns=['faultNumber'])

In [ ]:
df_output.value_counts()

In [7]:
from datetime import datetime

# CSV 파일로 저장
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"output_{current_time}.csv"
df_output.to_csv(filename)

In [ ]:
print(threshold)